# Encoder-Decoder with Attention

Encoder-Decoderモデルの発展系として、Attentionを追加してみます。  
ついでにEncocderのLSTMもBidirectional LSTMに変えてみましょう。

残念ながら、2017年10月8日時点で、Attentionを一行で追加してくれるような機能はChainerにはありません。  
Attentionの構造とChainerの関数などをきちんと理解し、自分で実装していくことになります。  

ライブラリのインポート、idの系列をembeddingの系列に変える関数は先と一緒です。

In [2]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

In [3]:
# 可変長単語id系列を可変長単語ベクトル系列へ写像する関数
def sequence_embed(embed, xs):
    x_len = [len(x) for x in xs]
    x_section = np.cumsum(x_len[:-1])
    ex = embed(F.concat(xs, axis=0))
    exs = F.split_axis(ex, x_section, 0)
    return exs

### Encoder

せっかくAttentionを使うので、EncoderのLSTMをBidirectionalにしてみます。  
Bidirectional LSTMは、すでにChainerに用意されているので、そこを変えるだけです。

In [4]:
class Encoder(Chain):
    def __init__(self, n_layers, n_source_vocab, n_units, dropout):
        super(Encoder, self).__init__()
        with self.init_scope():
            self.source_embed = L.EmbedID(in_size=n_source_vocab, out_size=n_units)
            
            # NStepLSTMをNStepBiLSTMに
            self.encoder_lstm = L.NStepBiLSTM(n_layers=n_layers, in_size=n_units,
                                              out_size=n_units, dropout=dropout)

            self.n_source_vocab = n_source_vocab
            self.n_units = n_units
    
    def __call__(self, source_xs):
        # 単語の系列を単語ベクトルへ
        exs = sequence_embed(self.source_embed, source_xs)
        
        # lstmの初期状態
        hx = None
        cx = None
        
        # lstmで各系列をエンコード
        hy, cy, ys = self.encoder_lstm(hx, cx, exs)
        return hy, cy, ys
        

注意としては、出力のhy, cyのshapeが変わります。  
n_layerのaxisが、forward LSTMとbackward LSTMの分を合わせ、n_layer * 2になります。  
また、ysの各隠れ層の次元数も2倍になります。  
よってDecoderのLSTMの隠れ層の次元数はEmbeddingやEncoderのn_unitsの二倍になります。

### Attention

Attentionを導入します。

今回はAttentionが機械翻訳に初めて適用された、 

・[Bahdanau et al. (2015) Neural Machine Translation by Jointly Learning Align and Translate](https://arxiv.org/pdf/1409.0473.pdf)  

のモデルを書いていきます。

<img src=https://raw.githubusercontent.com/kwashio/semi_tutorial/images/attention.png width=500px>

画像は[スタンフォード大学の授業のスライド](http://web.stanford.edu/class/cs224n/lectures/cs224n-2017-lecture10.pdf)から拝借しました。

Bahdanauのモデルでは、AttentionはLSTMの一番深い層（画像では一番上の層）で展開され、Decoderの隠れ状態を計算する際に使用されます。  
具体的には、$h_t$を計算する際に、

1. 一つ前の隠れ状態$h_{t-1}$からContext vector $c_t$を計算
1. $h_{t-1}$、$c_t$、input vector（画像だと$h_t$の下のベクトル）から$h_t$を計算

という風になります。

次のクラスAttentionでは、$h_{t-1}$から$a_t$を計算し、$c_t$を出力するまでの処理を記述しています。 

式にすると、Encoderのある隠れ状態の$score_i$は、

\begin{equation}
score_i = softmax(W_2 z_i) \\
z_i = tanh(W_1 (e_i \oplus h_{t-1}))
\end{equation}

ただし、$e_i$はEncoderの時点$i$における隠れ状態です。$\oplus$はベクトルの結合を表しています。

In [6]:
class Attention(Chain):
    def __init__(self, n_units):
        super(Attention, self).__init__()
        with self.init_scope():
            #eWとdWで上の式のW1を表している。
            # Encoder(BiLSTM)の隠れ状態の線形変換
            self.eW = L.Linear(n_units*2) # Decoderの隠れ層の次元数はEncoderのn_unitsの二倍
            
            # 一つ前のdecoder中間層の線形変換
            self.dW = L.Linear(n_units*2)
            
            
            # スコア計算用の線形変換、上の式のW2
            self.aW = L.Linear(1)
    
    def __call__(self, ehs, dh):
        # 各z_iの計算
        encoder_hidden = self.eW(ehs)
        
        # h_{t-1}の線形変換後のベクトルをbroadcastし、コピーしてencoder_hiddenに足し合わせる。
        decoder_hidden = F.broadcast_to(self.dW(dh), encoder_hidden.shape)
        attention_hidden = F.tanh(encoder_hidden + decoder_hidden)
        
        # scoreの計算。
        scores = F.softmax(self.aW(attention_hidden), axis=0)
        
        # context vectorの計算
        context = F.matmul(F.transpose(scores), ehs)
        # (1 , n_units*2)
        return context

### Decoder(with Attention)

Attentionを考慮したDecoderを書いていきます。  
少し複雑ですが、頑張ってついてきてください。  

Attentionなしの単純なEncoder-Decoderモデルを書いたときは、EncoderもDecoderもNStepLSTMで書くことができました。  
しかし、Attentionを考慮する場合は、Decoderの一番深いレイヤーの各隠れ状態を計算する際に、context vector $c_t$を計算に入れる必要があるため、単純にNStepLSTMを用いることはできません。

つまり、一番深いレイヤーの計算部分は自分で書かなければなりません。  
もう一度、さきほどの図を見てみましょう。

<img src=https://github.com/kwashio/semi_tutorial/blob/images/attention2.png?raw=true width=500px>


$c_t$を除く青い隠れ状態の部分はEncoderで計算済みです。  
赤色の隠れ状態はDecoderで計算するのですが、多層のLSTMを考えた時、赤枠の部分はNStepLSTMで計算できます。  
つまり、自分で書かなければいけないのは、Decoderのトップの層の部分ということになります。

Decoderもトップの層もLSTMなのですが、通常のLSTMとは異なり、$h_{t-1}$とinput以外にcontext vector $c_t$を考慮したLSTMです。  
このようなLSTMを記述する際は、レイヤーのLSTMやNStepLSTMではなく、[chainer.functions.lstm](https://docs.chainer.org/en/stable/reference/generated/chainer.functions.lstm.html#chainer.functions.lstm)を使います。

functionのLSTMは、大雑把に入力と出力を書くと、
```
c, h = lstm(previous_c, W1*previous_h + W2*input)
```
という風になっています。つまり、一つ前のcellと隠れ状態、input vectorを渡すと、新しいcellと隠れ状態を返してくれる関数です。  
注意点としては、previous_h、 input vectorはlstm関数に入力される前に、Linearレイヤー（W1, W2）により中間層の4倍の次元のベクトルに変換されなければないことです。  
なぜ4倍なのかというと、これはLSTMの各構成要素である、input gate、forget gate、output gate、new memory cellの計算に対応しています。

このlstm関数はレイヤーのLSTMと異なり、内部にパラメータを持っておらず、計算処理のみを担っています。  
LSTMとしてのパラメータは上の式における、W1とW2に相当します。

ざっくりとlstm関数を理解したところで、ここにcontext vector $c_t$を組み込みます。  
これは、以下のように行います。

```
c, h = lstm(previous_c, W1*previous_h + W2*input + W3*context)
```

これにより、context vectorを考慮しつつ、新たなcellと隠れ状態を計算することができます。  
では、Decoderクラスを書いていきましょう。
後々の処理過程を前に書いたAttentionなしのEncoder-Decoderモデルに合わせるため、出力はNStepLSTMと同じになるようにします。



In [7]:
class Decoder(Chain):
    
    def __init__(self, n_layers, n_target_vocab, n_units, attention, dropout):
        super(Decoder, self).__init__()
        with self.init_scope():
            self.n_target_vocab = n_target_vocab
            self.target_embed = L.EmbedID(n_target_vocab, n_units, dropout)
            self.n_layers = n_layers
            
            # layer数が１かそれ以上かで、NStepLSTMを使うかどうか分岐
            if self.n_layers > 1:
                self.pre_lstm = L.NStepLSTM(self.n_layers -1, n_units, n_units*2, dropout)
            
            # attention
            self.Att = attention
            
            self.dropout = dropout
            
            # topのLSTMの各線形変換（W1, W2, W3）
            # 中間層がn_units*2なので、それを４倍にする。
            self.lstm_input = L.Linear(n_units * 8)
            self.lstm_previous = L.Linear(n_units * 8)
            self.lstm_context = L.Linear(n_units * 8)
            
        
    def __call__(self, hy, cy, ys, target_xs):
        # hy, cy, ysはEncoder(BiLSTM)の出力
        # target_xsは、単語idの系列のリスト
        
        # attention以外の部分の計算
        batch_size = len(ys)
        
        # ターゲット言語の単語idの系列のリストを、embeddingの系列のリストへ
        exs = sequence_embed(self.target_embed, target_xs)
        
        # EncoderのBiLSTMのhy, cyのshapeをDecoderの構造に合わせる。
        # (n_layers*2, batchsize, n_units) -> (n_layers, batchsize, n_units*2)へ
        hy = F.reshape(hy, (self.n_layers, batch_size, -1))
        cy = F.reshape(cy, (self.n_layers, batch_size, -1))
        
        # n_layersが２以上のときは、NStepLSTMにより、一番深い層以外の隠れ状態を計算しておく。
        if self.n_layers > 1:
            unatt_n_layer = self.n_layers - 1
            pre_hy = hy[:unatt_n_layer]
            pre_cy = cy[:unatt_n_layer]
            after_h, after_c, pre_os = self.pre_lstm(pre_hy, pre_cy, exs)
        
        else:
            pre_os = exs
        # pre_osが、一番深い層のLSTMへのinputの系列になる。
        
        # 最終層の計算
        high_hy = hy[self.n_layers - 1]
        high_cy = cy[self.n_layers - 1]
        
        # NStepLSTMと出力を合わせるために、リストを３つ用意
        last_h = [] # 最後の隠れ状態のリスト
        last_c = [] # 最後のcellのリスト
        os = [] # 隠れ状態の系列のリスト
        
        # 各input系列ごとに処理
        for i, pre_eos in enumerate(pre_os):
            h = F.reshape(high_hy[i], (1,-1))
            c = F.reshape(high_hy[i], (1,-1))
            now_ys = ys[i] # 対応するEncoderの隠れ状態の系列 (lenght, n_units*2)
            temp_os = []
            
            # verticalにdropoutがかかるので、dropoutをかける場所はここ
            pre_eos = F.dropout(pre_eos, self.dropout)
            
            
            for x in pre_eos:
                # input vector
                x = F.reshape(x, (1,-1))
                
                # 一つ前の隠れ状態からcontext vectorを計算。
                context = self.Att(now_ys, h)
                
                # 次のセルと隠れ状態を計算する。
                c, h = F.lstm(c, self.lstm_input(x) + self.lstm_previous(h) + self.lstm_context(context))
                
                # 隠れ状態を保存
                temp_os.append(h)
            
            # 最後の隠れ状態、最後のセル、隠れ状態の系列を保存
            last_h.append(h)
            last_c.append(c)
            os.append(F.concat(temp_os, axis=0))
        
        # 出力をNStepLSTMに合わせるために、shapeを変換
        last_h = F.reshape(F.concat(last_h, axis=0), (1, batch_size, -1))
        last_c = F.reshape(F.concat(last_c, axis=0), (1, batch_size, -1))
        
        # n_layerが２以上のときは、NStepLSTMの出力とconcatする。
        if self.n_layers > 1:
            ho = F.concat([after_h, last_h], axis=0)
            co = F.concat([after_c, last_c], axis=0)
        else:
            ho = last_h
            co = last_c
            
        
        return ho, co, os

        

        
                

お疲れ様でした。  
頑張って出力の形を揃えたので、あとの処理はAttentionなしのEncoder-Decoderモデルとほぼ同じです。

In [17]:
UNK = 0
EOS = 1

In [18]:
class Encoder_Decoder_withAttention(Chain):
    
    def __init__(self, encoder, decoder):
        super(Encoder_Decoder_withAttention, self).__init__()
        with self.init_scope():
            self.encoder = encoder
            self.decoder = decoder
        
            self.W = L.Linear(self.decoder.n_target_vocab)
            
    def __call__(self, xs, ys):
        eos = self.xp.array([EOS], 'i')
        ys_in = [F.concat([eos, y], axis=0) for y in ys]
        ys_out = [F.concat([y, eos], axis=0) for y in ys]
        
        hy, cy, ys = self.encoder(xs)
        _, _, os = self.decoder(hy, cy, ys, ys_in)
        
        # loss calculation
        batch_size = len(xs)
        concat_os = F.concat(os, axis=0)
        concat_ys_out = F.concat(ys_out, axis=0)
        loss = F.sum(F.softmax_cross_entropy(
            self.W(concat_os), concat_ys_out, reduce='no')) / batch_size
        
        return loss
    
    def translate(self, xs, max_length = 100):
        batch_size = len(xs)
        
        with chainer.no_backprop_mode(), chainer.using_config('train', False):
            # Encode
            hy, cy, ys = self.encoder(xs)
            
            # decode時のinput用にbatch_size分のEOS=1を用意
            target_xs = self.xp.full((batch_size,1), 1, 'i')
            result = []
            
            ho = hy
            co = cy
            for i in range(max_length):
                ho, co, os = self.decoder(ho, co, ys, target_xs) 
                concat_os = F.concat(os, axis=0)
                wy = self.W(concat_os)
                target_xs = self.xp.argmax(wy.data, axis=1).astype('i')
                result.append(target_xs)
                target_xs = F.reshape(target_xs, (-1, 1)).data
            
            result = self.xp.stack(result).T
            # Remove EOS tags
            outs = []
            for y in result:
                inds = np.argwhere(y == EOS)
                if len(inds) > 0:
                    y = y[:inds[0, 0]]
                outs.append(y)
        return outs
            
            
            
            

Decoder  
target言語の系列のリストと、一つ前の隠れ状態、セルを受け取る。  
各タイムステップの隠れ状態を返す。

入力: xs = [np.array(1,3,4...), np.array(...),...], h, c  
出力: os = [(len, n_units), (len, n_units), ...]

h = (1, n_units by 2)
c = (1, n_units by 2)

In [19]:
xs = [np.random.randint(0, 50, size=np.random.randint(1, 15, 1)).astype('i')
      for i in range(20)]

In [20]:
xs

[array([ 4, 38,  5, 42], dtype=int32),
 array([35, 12, 26], dtype=int32),
 array([20, 17,  4], dtype=int32),
 array([ 4, 29, 19,  0, 46, 16, 45, 32, 27, 41], dtype=int32),
 array([20,  0], dtype=int32),
 array([44, 13], dtype=int32),
 array([36, 15], dtype=int32),
 array([23, 16], dtype=int32),
 array([43, 21, 15, 41, 23, 14, 39], dtype=int32),
 array([27, 34, 26], dtype=int32),
 array([17], dtype=int32),
 array([29, 48, 24,  0, 29, 48, 33, 27,  2], dtype=int32),
 array([16,  4, 11,  7, 29,  4, 34,  5], dtype=int32),
 array([39, 48, 37, 14, 32,  7,  5,  1, 44, 27, 43, 18, 17], dtype=int32),
 array([31,  3, 35, 16,  1, 26, 16, 14], dtype=int32),
 array([11, 11, 42,  8, 14, 47, 33, 46,  9], dtype=int32),
 array([40,  5, 18,  0,  8, 16, 32, 44, 22,  1, 30, 42,  8, 19], dtype=int32),
 array([20,  7, 32, 43, 21, 23, 36, 16,  4, 17, 31, 20, 11, 19], dtype=int32),
 array([47, 10,  6,  2, 24, 33, 48, 21, 32, 22,  4,  0], dtype=int32),
 array([8, 5, 4], dtype=int32)]

In [30]:
encoder = Encoder(1, 50, 100, 0)

In [31]:
hy, cy, ys = encoder(xs)

In [32]:
att = Attention(100)

In [33]:
decoder = Decoder(n_layers=1, n_target_vocab=100, n_units=100, attention=att, dropout=0)

In [34]:
decoder(hy, cy, ys, xs)[1].shape

(1, 20, 200)

In [35]:
eda = Encoder_Decoder_withAttention(encoder, decoder)

In [37]:
eda(xs,xs)

variable(34.225608825683594)